## 1. Wprowadzenie

### 1.1 Cel Projektu
Celem projektu było wytrenowanie modelu detekcji obiektów YOLO11n do automatycznej klasyfikacji śmieci na 6 kategorii:
- **BIODEGRADABLE** (biodegradowalne)
- **CARDBOARD** (tektura)
- **GLASS** (szkło)
- **METAL** (metal)
- **PAPER** (papier)
- **PLASTIC** (plastik)

### 1.2 Motywacja
Automatyczna klasyfikacja odpadów jest kluczowa dla efektywnego systemu recyklingu i zarządzania odpadami. Model wizyjny może wspierać sortowanie śmieci w czasie rzeczywistym.

### 1.3 Architektura YOLO11
YOLO (You Only Look Once) to rodzina modeli do detekcji obiektów typu "single-shot detector". Wersja YOLO11n (nano) to najmniejszy i najszybszy wariant, zoptymalizowany pod kątem niskiego zużycia zasobów.

## 2. Dataset

### 2.1 Źródło Danych
Dataset pochodzi z platformy **Roboflow**:
- **Nazwa projektu**: garbage-classification-3
- **Wersja**: 2
- **Licencja**: CC BY 4.0
- **URL**: https://universe.roboflow.com/material-identification/garbage-classification-3/dataset/2

### 2.2 Struktura Datasetu
Dataset podzielony na 3 zbiory:
- **Training set** (`train/images/`): ~1000+ obrazów
- **Validation set** (`valid/images/`): ~600+ obrazów
- **Test set** (`test/images/`): ~500+ obrazów

### 2.3 Format Anotacji
Format YOLO: każdy obraz ma odpowiadający plik `.txt` z etykietami w formacie:
```
<class_id> <x_center> <y_center> <width> <height>
```
gdzie współrzędne są znormalizowane do zakresu [0, 1].

## 3. Konfiguracja Treningu

### 3.1 Parametry Modelu
```yaml
Model: yolo11n.pt (pretrained)
Task: detect (object detection)
Device: mps (Apple Silicon GPU)
```

### 3.2 Hiperparametry Treningu
```yaml
Epochs: 30
Image size: 320x320 pixels
Batch size: -1 (auto)
Optimizer: AdamW
Learning rate (lr0): 0.01
Learning rate final (lrf): 0.01
Momentum: 0.937
Weight decay: 0.0005
Patience: 5 (early stopping)
```

### 3.3 Data Augmentation
```yaml
HSV-H: 0.015 (hue augmentation)
HSV-S: 0.7 (saturation)
HSV-V: 0.4 (value/brightness)
Translate: 0.1 (10% translation)
Scale: 0.5 (50% scale variation)
Horizontal flip: 0.5 (50% probability)
Mosaic: 1.0 (always enabled)
```

### 3.4 Validation Strategy
**Uwaga**: Parametr `val: false` oznacza, że walidacja była przeprowadzona **tylko po zakończeniu wszystkich epok**, nie po każdej epoce z osobna.

## 4. Implementacja

In [ ]:
# Instalacja bibliotek (jeśli potrzebne)
# !pip install ultralytics pandas matplotlib seaborn

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml

# Konfiguracja wykresów
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

ModuleNotFoundError: No module named 'pandas'

### 4.1 Kod Treningu (uproszczony)

In [ ]:
from ultralytics import YOLO

# Uwaga: Ten kod pokazuje strukturę, faktyczny trening był już wykonany
# z parametrami zapisanymi w args.yaml

# Load pretrained YOLO11n model
# model = YOLO("yolo11n.pt")

# Train the model
# train_results = model.train(
#     data="data.yaml",
#     epochs=30,
#     imgsz=320,
#     device="mps",
#     optimizer="AdamW",
#     lr0=0.01,
#     patience=5
# )

# Validate
# metrics = model.val()

# Export to ONNX
# model.export(format="onnx")

## 5. Analiza Wyników Treningu

In [ ]:
# Wczytanie wyników z pliku CSV
results_path = r"c:\Users\fnubu\Documents\code\STUDIA\PRIAD\GarbageClassification\GARBAGE CLASSIFICATION 3\runs\detect\train\results.csv"
df = pd.read_csv(results_path)

# Wyświetlenie pierwszych i ostatnich epok
print("=== Pierwsze 3 epoki ===")
print(df[['epoch', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss']].head(3))
print("\n=== Ostatnie 3 epoki ===")
print(df[['epoch', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss']].tail(3))

### 5.1 Progres Loss Functions

In [ ]:
# Wykres strat treningowych
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Box Loss
axes[0].plot(df['epoch'], df['train/box_loss'], marker='o', linewidth=2, color='#2E86AB')
axes[0].set_title('Box Loss (Localization)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].grid(True, alpha=0.3)

# Classification Loss
axes[1].plot(df['epoch'], df['train/cls_loss'], marker='s', linewidth=2, color='#A23B72')
axes[1].set_title('Classification Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].grid(True, alpha=0.3)

# DFL Loss
axes[2].plot(df['epoch'], df['train/dfl_loss'], marker='^', linewidth=2, color='#F18F01')
axes[2].set_title('DFL Loss (Distribution Focal Loss)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Loss')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_losses.png', dpi=300, bbox_inches='tight')
plt.show()

### 5.2 Interpretacja Strat

**Box Loss (Localization Loss)**:
- Epoka 1: 1.688 → Epoka 30: 1.252
- Spadek o ~26% - model poprawia lokalizację bounding boxes

**Classification Loss**:
- Epoka 1: 2.344 → Epoka 30: 1.334
- Spadek o ~43% - najlepszy progres, model uczy się rozróżniać klasy

**DFL Loss (Distribution Focal Loss)**:
- Epoka 1: 1.505 → Epoka 30: 1.275
- Spadek o ~15% - model precyzuje przewidywania rozkładów

✅ **Wniosek**: Wszystkie straty maleją konsekwentnie - model się uczył poprawnie.

### 5.3 Learning Rate Schedule

In [ ]:
# Wykres learning rate
plt.figure(figsize=(10, 5))
plt.plot(df['epoch'], df['lr/pg0'], marker='o', linewidth=2, color='#06A77D', label='Learning Rate')
plt.title('Learning Rate Schedule', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('learning_rate.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Learning rate początkowy: {df['lr/pg0'].iloc[0]:.6f}")
print(f"Learning rate końcowy: {df['lr/pg0'].iloc[-1]:.6f}")

## 6. Metryki Walidacyjne

### 6.1 Ostateczne Wyniki (Epoka 30)

In [ ]:
# Wyciągnięcie metryk z ostatniej epoki (jedyna z walidacją)
final_metrics = df.iloc[-1]

metrics_dict = {
    'Metric': ['Precision', 'Recall', 'mAP@0.5', 'mAP@0.5:0.95', 'Val Box Loss', 'Val Cls Loss', 'Val DFL Loss'],
    'Value': [
        final_metrics['metrics/precision(B)'],
        final_metrics['metrics/recall(B)'],
        final_metrics['metrics/mAP50(B)'],
        final_metrics['metrics/mAP50-95(B)'],
        final_metrics['val/box_loss'],
        final_metrics['val/cls_loss'],
        final_metrics['val/dfl_loss']
    ]
}

metrics_df = pd.DataFrame(metrics_dict)
print("\n" + "="*50)
print("OSTATECZNE METRYKI WALIDACYJNE (Epoka 30)")
print("="*50)
print(metrics_df.to_string(index=False))
print("="*50)

In [ ]:
# Wizualizacja metryk
fig, ax = plt.subplots(figsize=(10, 6))

metrics_to_plot = metrics_df.iloc[:4]  # Tylko precision, recall, mAP50, mAP50-95
colors = ['#2E86AB', '#A23B72', '#F18F01', '#06A77D']

bars = ax.barh(metrics_to_plot['Metric'], metrics_to_plot['Value'], color=colors)
ax.set_xlabel('Score', fontsize=12)
ax.set_title('Validation Metrics (Epoch 30)', fontsize=14, fontweight='bold')
ax.set_xlim(0, 1)

# Dodanie wartości na końcu słupków
for i, (metric, value) in enumerate(zip(metrics_to_plot['Metric'], metrics_to_plot['Value'])):
    ax.text(value + 0.02, i, f'{value:.3f}', va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('validation_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

### 6.2 Interpretacja Metryk

**Precision: 0.282 (28.2%)**
- Z wszystkich obiektów, które model wykrył, tylko 28% było prawidłowo sklasyfikowanych
- **Problem**: Dużo false positives (model "widzi" śmieci tam, gdzie ich nie ma lub błędnie je klasyfikuje)

**Recall: 0.082 (8.2%)**
- Model znalazł tylko 8% wszystkich obiektów obecnych w obrazach
- **Problem**: Bardzo dużo false negatives (model pomija większość śmieci)

**mAP@0.5: 0.084 (8.4%)**
- Mean Average Precision przy IoU threshold 0.5
- Wartość poniżej 0.1 = **bardzo słaba wydajność**
- Standardowo oczekiwalibyśmy mAP50 > 0.5 dla przyzwoitego modelu

**mAP@0.5:0.95: 0.057 (5.7%)**
- Średnia z mAP przy progach IoU od 0.5 do 0.95
- Jeszcze niższa wartość potwierdza słabą lokalizację obiektów

⚠️ **Podsumowanie**: Model wymaga znaczącej poprawy - obecne wyniki są poniżej użytecznego poziomu.

## 7. Dyskusja i Analiza Problemów

### 7.1 Zidentyfikowane Problemy

#### 1. **Słabe Wyniki Walidacyjne**
Główny problem: mAP@0.5 = 8.4% to wynik nieużyteczny w praktyce.

**Możliwe przyczyny**:

**a) Zbyt mały rozmiar obrazów (320px)**
- Standardowy rozmiar dla YOLO to 640px
- 320px = 4x mniej pikseli = utrata szczegółów obiektów
- Szczególnie problematyczne dla małych obiektów

**b) Model Nano (YOLO11n)**
- Najmniejszy wariant YOLOv11
- Szybki, ale najmniej dokładny
- Lepsze wyniki: YOLO11s (small), YOLO11m (medium) lub YOLO11l (large)

**c) Za mało epok (30)**
- Training loss wciąż spadał na epoce 30
- Model prawdopodobnie nie osiągnął pełnej zbieżności
- Rekomendacja: 100-200 epok dla złożonych datasetsów

**d) Brak walidacji międzyepokowej**
- `val: false` = brak monitoringu podczas treningu
- Nie można było obserwować overfittingu
- Brak informacji o najlepszym checkpoincie

**e) Możliwa niezrównoważona dystrybucja klas**
- W katalogu `train/images/` zauważono przewagę plików z klasą PLASTIC
- Niezbalansowany dataset może powodować bias modelu

#### 2. **Rozbieżność Kodu i Faktycznych Parametrów**
- Plik `model.py` pokazuje: `epochs=1, imgsz=640, device="cpu"`
- Faktyczne parametry (z `args.yaml`): `epochs=30, imgsz=320, device="mps"`
- **Wniosek**: Kod w `model.py` to tylko przykład, faktyczny trening był uruchomiony z innych parametrów (prawdopodobnie CLI lub inny skrypt)

### 7.2 Rekomendacje na Przyszłość

**Aby poprawić wyniki, zalecam**:

1. **Zwiększyć rozmiar obrazów do 640px**
   ```python
   model.train(data="data.yaml", epochs=100, imgsz=640, ...)
   ```

2. **Użyć większego modelu (YOLO11s lub YOLO11m)**
   ```python
   model = YOLO("yolo11s.pt")  # lub yolo11m.pt
   ```

3. **Wydłużyć trening do 100-200 epok**
   ```python
   epochs=100
   ```

4. **Włączyć walidację po każdej epoce**
   ```python
   val=True  # w parametrach treningu
   ```

5. **Sprawdzić balans klas w datasecie**
   ```python
   # Analiza dystrybucji klas
   from collections import Counter
   import glob
   
   labels = []
   for file in glob.glob('train/labels/*.txt'):
       with open(file) as f:
           labels.extend([int(line.split()[0]) for line in f])
   
   print(Counter(labels))
   ```

6. **Rozważyć class weights dla niezbalansowanych klas**

7. **Użyć większego learning rate warmup**
   ```python
   warmup_epochs=5  # zamiast 3
   ```

8. **Eksperymentować z augmentacjami**
   - Możliwe zwiększenie `mosaic`, `mixup`, `copy_paste`

## 8. Wnioski

### 8.1 Podsumowanie Treningu

✅ **Pozytywne aspekty**:
- Model został pomyślnie wytrenowany przez 30 epok
- Training loss konsekwentnie malał we wszystkich składowych
- Infrastruktura treningu działa poprawnie (MPS, augmentacje)
- Dataset jest poprawnie sformatowany (Roboflow YOLO format)
- Eksport do ONNX dostępny dla deployment

❌ **Problemy do rozwiązania**:
- **Bardzo słabe metryki walidacyjne** (mAP@0.5 = 8.4%)
- Recall 8.2% = model pomija 92% obiektów
- Model wymaga gruntownej poprawy przed użyciem praktycznym
- Możliwa niezrównoważona dystrybucja klas w datasecie

### 8.2 Dalsze Kroki

**Krótkoterminowe (następny trening)**:
1. Zwiększyć rozmiar obrazów do 640px
2. Użyć YOLO11s zamiast YOLO11n
3. Trenować przez 100 epok z walidacją co epokę
4. Monitorować metryki i zapisywać checkpointy

**Średnioterminowe (optymalizacja)**:
1. Analiza datasetu: rozkład klas, jakość anotacji
2. Eksperyment z różnymi augmentacjami
3. Transfer learning z modeli trenowanych na podobnych datasetsach
4. Ensemble z większymi modelami

**Długoterminowe (deployment)**:
1. Osiągnąć mAP@0.5 > 0.5 przed wdrożeniem
2. Testy w realnych warunkach
3. Continuous training z nowymi danymi
4. Monitoring production metrics

### 8.3 Nauka z Projektu

Ten projekt pokazał:
- **Znaczenie hiperparametrów**: Rozmiar obrazów i wielkość modelu mają kluczowy wpływ
- **Potrzebę monitoringu**: Walidacja tylko na końcu nie pozwala na wczesne wykrycie problemów
- **Wartość datasetu**: Jakość i balans danych jest fundamentalna
- **Iteracyjność deep learning**: Pierwszy model rzadko jest optymalny

---

**Model wymaga retraininingu z poprawionymi parametrami przed użyciem w produkcji.**

## 9. Bibliografia i Źródła

1. **Ultralytics YOLOv11 Documentation**  
   https://docs.ultralytics.com/

2. **Roboflow Dataset**  
   https://universe.roboflow.com/material-identification/garbage-classification-3/dataset/2

3. **YOLOv11 Paper** (jeśli dostępny)  
   [Link do publikacji]

4. **Mean Average Precision (mAP) Explained**  
   [Dodaj źródło teoretyczne]

5. **Object Detection Metrics Guide**  
   [Dodaj źródło]

---

**Appendix: Pliki Projektu**
- `data.yaml`: Konfiguracja datasetu
- `args.yaml`: Kompletne parametry treningu
- `results.csv`: Historia metryk treningowych
- `weights/best.pt`: Najlepszy checkpoint modelu
- `weights/last.pt`: Ostatni checkpoint (epoka 30)
- `weights/best.onnx`: Model w formacie ONNX do deploymentu